In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from tabulate import tabulate

import os, shutil, time, pdb, random
import scipy.stats as stats 
import scipy

from math import pi
from datetime import datetime
from collections import OrderedDict
import pickle

import torch
from torch.utils.data import TensorDataset, DataLoader

from importlib import reload
from models import *
from utils import *
from runmanager import *
from experiment import *
from plot_utils import *
from preprocessing_utils import *
from analysis_seasonal import * 

from sklearn.metrics import mean_squared_error as mse

import matplotlib
matplotlib.rc_file_defaults()
%matplotlib inline

import CONFIG

pd.options.display.max_columns = None

np.random.seed(4)

%load_ext autoreload
%autoreload 2

device = CONFIG.device

### Import data

In [12]:
data = DataPreprocessing(train_path=CONFIG.TRAIN_PATH, start=CONFIG.start, end=CONFIG.end, 
                add_yesterday = False, 
                basin_filter = None, 
                split_bias_corrected_only = False, 
                filter_incomplete_years = False, 
                include_non_bc_stations = True, 
                split_by = 'station')

### Split data into held out sets for K-fold cross validation

In [13]:
data.split_stations()

### Prepare data

In [14]:
# CONFIG.predictors.append('obs_yesterday')
data.input_data(CONFIG.predictors, CONFIG.predictand, sort_by_quantile=CONFIG.sort_by_quantile)

/users/marron31/repos/upper-indus-prec-bc/preprocessing_utils.py:205: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  QM_df = QM_df.append(QM_data[s])
/users/marron31/repos/upper-indus-prec-bc/preprocessing_utils.py:205: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  QM_df = QM_df.append(QM_data[s])
/users/marron31/repos/upper-indus-prec-bc/preprocessing_utils.py:205: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  QM_df = QM_df.append(QM_data[s])
/users/marron31/repos/upper-indus-prec-bc/preprocessing_utils.py:205: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  QM_df = QM_df.append(QM_data[s])
/users/marron31/repos/upper-indus-prec-bc/prepro

## Multi-Run: Train model with different hyperparameters

### Model run

In [15]:
st_test, predictions, importances = multirun(data, CONFIG.predictors, CONFIG.params, CONFIG.epochs, 
                                split_by = 'station', sequential_samples = False, load_run = None, experiment_label='SUSHIWAT')

,run,epoch,loss,valid_loss,test_loss,decision_loss,epoch duration,run duration,lr,batch_size,likelihood_fn,dropout_rate,k,model_arch
0,1,1,0.969458,1.896931,2.332096,1.896931,1.131365,1.136512,0.005,128,bgmm,0,0,"(VGLM, [])"
1,1,2,1.030030,1.959911,2.541596,1.959911,1.059794,2.405536,0.005,128,bgmm,0,0,"(VGLM, [])"
2,1,3,1.079324,1.995050,2.690020,1.995050,1.011632,3.446845,0.005,128,bgmm,0,0,"(VGLM, [])"
3,1,4,1.221408,1.663102,2.928188,1.663102,1.027108,4.499626,0.005,128,bgmm,0,0,"(VGLM, [])"
4,1,5,1.168919,2.618845,3.449633,2.618845,1.013980,5.555817,0.005,128,bgmm,0,0,"(VGLM, [])"
5,1,6,1.415992,2.093991,inf,2.093991,1.010513,6.606770,0.005,128,bgmm,0,0,"(VGLM, [])"
6,1,7,1.626374,1.941715,inf,1.941715,0.975957,7.614655,0.005,128,bgmm,0,0,"(VGLM, [])"
7,1,8,2.603916,1.938419,inf,1.938419,1.011184,8.656074,0.005,128,bgmm,0,0,"(VGLM, [])"
8,1,9,1.306061,1.540836,inf,1.540836,1.022666,9.712116,0.005,128,bgmm,0,0,"(VGLM, [])"
9,1,10,1.133800,1.845853,inf,1.845853,1.008887,10.773441,0.005,128,bgmm,0,0,"(VGLM, [])"


KeyboardInterrupt: 

In [8]:
sample_cols = [f'sample_{i}' for i in range(CONFIG.n_samples)]
add_cols = []
columns = ['Prec','wrf_prcp','wrf_bc_prcp','precip_norris']

In [9]:
# with open('_experiments/None/predictions.pkl', 'rb') as handle:
#     b = pickle.load(handle)

# with open('_experiments/magali/predictions.pkl', 'rb') as handle:
#     a = pickle.load(handle)
    
# a.update(b)
# a.update(predictions)

# predictions = a.copy()

In [10]:
for p in predictions.keys(): 
    for k,v in predictions[p].items():
        v['wrf_prcp'] = v['precip_norris'] 
        v['wrf_bc_prcp'] = v['precip_norris'] 

In [11]:
for k,v in predictions.items():
    print(f"{k : <40} {v['k_all'].BS.mean():.3f}")

VGLM_[]_bgmm_B=128_D=0                   0.574
MLP_[10]_bgmm_B=128_D=0                  0.589
SimpleRNN_[10]_bgmm_B=128_D=0            0.427


In [12]:
from utils import QS

In [13]:
def compute_QS(predictions, quantile = 0.5): 
    for k,v in predictions.items(): 
        p = v['k_all']
        p[f'QS_quantile_{quantile}'] = quantile
        likelihood = k.split('_')[2]
        p[f'QS_sample_{quantile}'] = p.apply(sample_apply, axis=1, args=(likelihood, 10000, f'QS_quantile_{quantile}'))
        p[f'QS_{quantile}'] = p.apply(QS, axis=1, args=('QS_sample', 'Prec', quantile))
        print(f'Completed {k} {quantile}')

In [14]:
compute_QS(predictions, quantile = 0.10)
compute_QS(predictions, quantile = 0.25)
compute_QS(predictions, quantile = 0.50)
compute_QS(predictions, quantile = 0.75)
compute_QS(predictions, quantile = 0.90)
compute_QS(predictions, quantile = 0.95)

Completed VGLM_[]_bgmm_B=128_D=0 0.1
Completed MLP_[10]_bgmm_B=128_D=0 0.1
Completed SimpleRNN_[10]_bgmm_B=128_D=0 0.1
Completed VGLM_[]_bgmm_B=128_D=0 0.25
Completed MLP_[10]_bgmm_B=128_D=0 0.25
Completed SimpleRNN_[10]_bgmm_B=128_D=0 0.25
Completed VGLM_[]_bgmm_B=128_D=0 0.5
Completed MLP_[10]_bgmm_B=128_D=0 0.5
Completed SimpleRNN_[10]_bgmm_B=128_D=0 0.5
Completed VGLM_[]_bgmm_B=128_D=0 0.75
Completed MLP_[10]_bgmm_B=128_D=0 0.75
Completed SimpleRNN_[10]_bgmm_B=128_D=0 0.75
Completed VGLM_[]_bgmm_B=128_D=0 0.9
Completed MLP_[10]_bgmm_B=128_D=0 0.9
Completed SimpleRNN_[10]_bgmm_B=128_D=0 0.9
Completed VGLM_[]_bgmm_B=128_D=0 0.95
Completed MLP_[10]_bgmm_B=128_D=0 0.95
Completed SimpleRNN_[10]_bgmm_B=128_D=0 0.95


In [ ]:
# for k,v in predictions.items():
#     print(f"{k : <40} {v['k_all']['QS_0.1'].mean():.3f}")

In [ ]:
# for k,v in predictions.items():
#     print(f"{k : <40} {v['k_all']['QS_0.25'].mean():.3f}")

In [ ]:
# for k,v in predictions.items():
#     print(f"{k : <40} {v['k_all']['QS_0.5'].mean():.3f}")

In [ ]:
# for k,v in predictions.items():
#     print(f"{k : <40} {v['k_all']['QS_0.75'].mean():.3f}")

In [ ]:
# for k,v in predictions.items():
#     print(f"{k : <40} {v['k_all']['QS_0.9'].mean():.3f}")

In [ ]:
# for k,v in predictions.items():
#     print(f"{k : <40} {v['k_all']['QS_0.95'].mean():.3f}")

In [21]:
table_of_predictions_ks_test(predictions, CONFIG.seasons, columns, sample_cols, add_cols)

VGLM_[]_bgmm_B=128_D=0
MLP_[10]_bgmm_B=128_D=0
SimpleRNN_[10]_bgmm_B=128_D=0
Model                          JFM mean    JFM median    AM mean    AM median    JJAS mean    JJAS median    OND mean    OND median
-----------------------------  ----------  ------------  ---------  -----------  -----------  -------------  ----------  ------------
Bann                           0.0757      0.0573        0.1029     0.0567       0.1522       0.0601         0.1348      0.0584
BannCorr                       0.0757      0.0573        0.1029     0.0567       0.1522       0.0601         0.1348      0.0584
Norr                           0.0757      0.0573        0.1029     0.0567       0.1522       0.0601         0.1348      0.0584
VGLM_[]_bgmm_B=128_D=0         0.2198      0.0585        0.1215     0.0920       0.0566       0.0509         0.1680      0.0583
MLP_[10]_bgmm_B=128_D=0        0.2107      0.0607        0.2306     0.0692       0.0548       0.0246         0.1978      0.0492
SimpleRNN_[10]_bg

In [22]:
table_of_predictions_for_metric(predictions, CONFIG.seasons, columns, CONFIG.n_samples, sample_cols, add_cols, metric = 'smape', prefix='smape')

Model                          JFM mean    JFM median    AM mean    AM median    JJAS mean    JJAS median    OND mean    OND median
-----------------------------  ----------  ------------  ---------  -----------  -----------  -------------  ----------  ------------
Prec                           0.00        0.00          0.00       0.00         0.00         0.00           0.00        0.00
wrf_prcp                       0.23        0.18          0.38       0.35         0.21         0.18           0.39        0.34
wrf_bc_prcp                    0.23        0.18          0.38       0.35         0.21         0.18           0.39        0.34
precip_norris                  0.23        0.18          0.38       0.35         0.21         0.18           0.39        0.34
VGLM_[]_bgmm_B=128_D=0         0.25        0.21          0.36       0.32         0.20         0.15           0.40        0.30
MLP_[10]_bgmm_B=128_D=0        0.24        0.18          0.29       0.25         0.20         0.16      

In [23]:
table_of_predictions_for_metric(predictions, CONFIG.seasons, columns, CONFIG.n_samples, sample_cols, add_cols, metric = 'edd', prefix='edd')

Model                          JFM mean    JFM median    AM mean    AM median    JJAS mean    JJAS median    OND mean    OND median
-----------------------------  ----------  ------------  ---------  -----------  -----------  -------------  ----------  ------------
Prec                           0.00        0.00          0.00       0.00         0.00         0.00           0.00        0.00
wrf_prcp                       14.21       10.00         12.79      12.00        32.95        34.00          7.52        5.50
wrf_bc_prcp                    14.21       10.00         12.79      12.00        32.95        34.00          7.52        5.50
precip_norris                  14.21       10.00         12.79      12.00        32.95        34.00          7.52        5.50
VGLM_[]_bgmm_B=128_D=0         5.74        4.60          5.85       4.30         10.32        7.70           5.16        4.30
MLP_[10]_bgmm_B=128_D=0        5.86        4.80          4.77       3.70         10.84        8.80      

In [24]:
table_of_predictions_for_metric(predictions, CONFIG.seasons, columns, CONFIG.n_samples, sample_cols, add_cols, metric = 'ae', prefix='ae')

Model                          JFM mean    JFM median    AM mean    AM median    JJAS mean    JJAS median    OND mean    OND median
-----------------------------  ----------  ------------  ---------  -----------  -----------  -------------  ----------  ------------
Prec                           0.00        0.00          0.00       0.00         0.00         0.00           0.00        0.00
wrf_prcp                       74.51       55.94         53.95      44.58        293.28       191.28         33.85       19.84
wrf_bc_prcp                    74.51       55.94         53.95      44.58        293.28       191.28         33.85       19.84
precip_norris                  74.51       55.94         53.95      44.58        293.28       191.28         33.85       19.84
VGLM_[]_bgmm_B=128_D=0         84.24       63.04         51.28      37.37        273.26       203.65         44.41       37.07
MLP_[10]_bgmm_B=128_D=0        86.69       62.62         45.33      37.92        276.16       187.20

In [49]:
# table_of_predictions_for_metric(predictions, CONFIG.seasons, columns, CONFIG.n_samples, sample_cols, add_cols, metric = 'se', prefix='se')

In [ ]:
a = pd.read_csv('results.csv')

b = a.groupby(['k','run']).agg({'valid_loss': 'min', 
                                 'model_arch': 'first', 
                                 'likelihood_fn': 'first',
                                 'lr':'first',
                                 'batch_size':'first',
                                 'dropout_rate':'first',
                               })

c = b.groupby(['run']).agg({'valid_loss': 'mean', 
                        'model_arch': 'first', 
                        'likelihood_fn': 'first',
                        'lr':'first',
                        'batch_size':'first',
                        'dropout_rate':'first',
                         }
                   ).sort_values('valid_loss').reset_index()

c.groupby(['model_arch',
           'likelihood_fn',
           'lr',
           'batch_size',
           'dropout_rate',
           ]).agg({'valid_loss': 'mean'}
                   ).sort_values('valid_loss').reset_index()